# Math5671 Group3 Assignment 02

**Hukai Luo, Tae Park, Xuanbo Huang**

In this assignment, we will rewrite the sample code using TensorFlow based on the example. Try different values with the parameters for better results.

## Part1

In [1]:
import gym
import random
import math
import numpy as np
import tensorflow as tf

env = gym.make('FrozenLake-v0')

Q = tf.Variable(tf.zeros([env.observation_space.n,env.action_space.n]),  dtype=tf.float32)
sess = tf.Session()
sess.run(tf.global_variables_initializer())
#discount factor
gamma = tf.constant(0.8)

#learning rate
alpha = tf.constant(0.1)

#total number of episodes you want to play during training
episodes = 100
total = tf.constant(0.0)

C:\Users\User\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Import several packages and set learning parameters
## Part2

In [2]:
for i in range(episodes):
    state = env.reset()
    done = True
    epsilon = tf.divide(1.0,1+i)
    
    while done:
        
        def f1(): return tf.convert_to_tensor(env.action_space.sample(), preferred_dtype=tf.int64)
        def f2(): return tf.argmax(Q[state], output_type=tf.int64)
        action = tf.cond(epsilon < tf.random_uniform((),0,1),true_fn = f1,false_fn = f2)
        new_state, reward, done, info = env.step(int(sess.run(action)))
    
    
        new_state, reward, done, info = env.step(action)
        # step(self, action): Step the environment by one timestep. Returns observation, reward, done, info.
        
        # Update Q-Table
        # Q[state,action] = (1-alpha)*Q[state,action] + alpha*(reward + gamma * np.max(Q[new_state]))
        X = tf.multiply(gamma,tf.reduce_max(Q[new_state]))  
        Y = tf.add(X,reward)
        Z = tf.subtract(Y,Q[state,action])
        W = tf.multiply(alpha,Z)
        V = tf.add(Q[state, action], W)
        
        #update the new value to q-table
        update = tf.assign(Q[state, action],V)
        sess.run(update)
        state = new_state
        total = tf.add(total,reward)
        
print("Average rewards: "+str(sess.run(tf.divide(total,episodes))))
print("Learned Q-Table")
print(sess.run(Q))


KeyError: <tf.Tensor 'cond/Merge:0' shape=() dtype=int64>

In [ ]:
import gym
import numpy as np
import random
import math
import tensorflow as tf

env = gym.make('FrozenLake-v0')

Q = tf.Variable(tf.random_uniform([env.observation_space.n,env.action_space.n],0,0),  dtype=tf.float32)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

#discount factor
gamma = tf.constant(0.8)

#learning rate
alpha = tf.constant(0.1)

#total number of episodes you want to play during training
episodes = 1000

total = tf.constant(0, dtype=tf.float32)
i = 0


for i in range(episodes):
    state = env.reset()
    done = True
    epsilon = tf.divide(1.0,1+i)
    
    while done:
        
        def f1(): return tf.convert_to_tensor(env.action_space.sample(), preferred_dtype=tf.int64)
        def f2(): return tf.argmax(Q[state], output_type=tf.int64)
        action = tf.cond(epsilon < tf.random_uniform((),0,1),true_fn = f1,false_fn = f2)
        new_state, reward, done, info = env.step(int(sess.run(action)))

        # Update Q-Table
        # Q[state,action] = (1-alpha)*Q[state,action] + alpha*(reward + gamma * np.max(Q[new_state]))
        X = tf.multiply(gamma,tf.reduce_max(Q[new_state]))  
        Y = tf.add(X,reward)
        Z = tf.subtract(Y,Q[state,action])
        W = tf.multiply(alpha,Z)
        V = tf.add(Q[state, action], W)
        
        #update the new value to q-table
        update = tf.assign(Q[state, action],V)
        sess.run(update)
        state = new_state
        total = tf.add(total,reward)
        
print("Average rewards: " + str(sess.run(tf.divide(total,episodes))))
print("Learned Q-Table")
print(sess.run(Q))

In order to make the Q-table useful and efficient, the parameter 'episodes' should be large enough(more than 5000). However, the program running slowly, it may take several hours to run it. This program will give poor performance because it creates several new TensorFlow graph nodes per operation. The underlying assumption in TensorFlow is (approximately) that we'll build a graph once and then call sess.run() on (various parts of) it multiple times. The first time we run a graph is relatively expensive, because TensorFlow has to build various data structures and optimize the execution of the graph across multiple devices.

$\alpha$ is the learning rate, $\gamma$ is the discount factor that controls the importance of future rewards. $Q(s_t,a_t)$ is the Q-table value of action a t in state s t and time t. In the assignment, we try defferent values with parameters for better results. We set $\alpha=0.5,\gamma=0.1,episodes=10000$, get average reward=0.0147; we also set $\alpha=0.9,\gamma=0.1,episodes=10000$, get average reward=0.0161; set $\alpha=0.6,\gamma=0.1,episodes=10000$, get average reward=0.0163;set $\alpha=0.8,\gamma=0.5,episodes=10000$, get average reward=0.0134.

**If the program runs for a very long time, just set the episodes = 10, it will save a lot of time, but the Q-table results may not be very good.**
# Part3

In [3]:
from IPython.display import clear_output
import time
env.reset()
episodes = tf.constant(10)
max_steps = tf.constant(1000)
episode = 0

for episode in range(sess.run(episodes)):
    state = env.reset()
    step = 0
    done = False
    clear_output()
    print("*****************")
    print("Episodes", episode)
    print("*****************")
    
    for step in range(sess.run(max_steps)):        
        # Take the action that has the best Q value
        action = int(sess.run(tf.argmax(Q[state])))      
        new_state, reward, done, info = env.step(action)
        
        if done:
            # Only print the last state (to see if our agent is on the goal or fall into an hole)
            env.render()
            print("*****************")
            print("Steps", step)
            print("*****************")
            time.sleep(2)
            break
        state = new_state
        
env.close()

*****************
Episodes 9
*****************
  (Left)
SFFF
FHFH
FFFH
HFFG
*****************
Steps 14
*****************


In this part we will take the action that has the best Q value and only print the last state (to see if our agent is on the goal or fall into an hole).